In [3]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# 데이터셋 로드
iris = load_iris()
x = iris.data[:,2:] # 꽃잎의 길이와 너비
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3, 
                                                    random_state=2021, shuffle=True)
# 약한 학습기 구축
log_model = LogisticRegression()
rnd_model = RandomForestClassifier()
svm_model = SVC()

# 앙상블 모델 구축
voting_model = VotingClassifier(
    estimators = [('lr', log_model),
                  ('rf', rnd_model),
                  ('svc',svm_model)],
                  voting = 'hard' # 직접투표 방법
)
# 앙상블 모델 학습
voting_model.fit(x_train, y_train)

# 모델 비교
for model in (log_model,rnd_model,svm_model,voting_model):
  model.fit(x_train,y_train)
  y_pred = model.predict(x_test)
  print(model.__class__.__name__," : ",accuracy_score(y_test,y_pred))

LogisticRegression  :  1.0
RandomForestClassifier  :  0.9333333333333333
SVC  :  1.0
VotingClassifier  :  1.0


In [10]:
# 사이킷 런에서 배깅과 페이스팅

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# 모델 구축
bag_model = BaggingClassifier(
    DecisionTreeClassifier(), # 약한 학습기 (결정 트리)
    n_estimators=500, # 약한 학습기 (결정 트리) 500개 +> 첫 두 줄이 랜덤 포레스트 쓴다는 뜻
    max_samples=0.05, # 0~1 사이의 실수선택 (실수 * 샘플 수)
    bootstrap=True,
    n_jobs=-1 # +> 훈련에 사용할 CPU 코어의 수, -1 쓰면 컴퓨터의 모든 코어 사용
)
# 모델 학습
bag_model.fit(x_train, y_train)

# 모델 예측
y_pred = bag_model.predict(x_test)

# 모델 평가
print(bag_model.__class__.__name__, " : ", accuracy_score(y_test,y_pred))

BaggingClassifier  :  0.9777777777777777


In [12]:
# random forest 방식
# 전체 특성 중에서 최선의 특성을 찾는 것이 아니라.
# 무작위로 선택한 특성들 중에서 최선의 특성을 찾는 방식을
# 채택하여 무작위성을 가지게 된다.
# 분산을 전체적으로 낮추어 더 훌륭한 모델을 만든다.

from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 모델 구축
rnd_model = RandomForestClassifier(
    n_estimators = 500,
    max_leaf_nodes = 16,
    n_jobs = -1
)

# 모델 학습
rnd_model.fit(x_train, y_train)

# 모델 예측
y_predrf = rnd_model.predict(x_test)

# 모델 평가
print("rnd_model : ", accuracy_score(y_predrf, y_test))

rnd_model :  0.9555555555555556


In [15]:
from sklearn.ensemble import AdaBoostClassifier # => 잘 학습하지 못하는 것에 대해서 가중치를 두겠다.
from sklearn.tree import DecisionTreeClassifier

# 아다부스트 모델 구축
# 아다부스트의 학습기 : Decision Tree (max depth = 1 ) 사용
# 학습기 개수 (n_estimators) : 200 개
# SAMME(Stagewise Additive Modeling using a Multiclass Exponential loss function) 알고리즘 사용
# 기본 학습기가 확률 추정(predict proba) 이 가능하면 SAMME.R 사용 => 일반적으로 성능이 더 좋음
ada_model = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators=200,
    algorithm='SAMME.R',
    learning_rate=0.5
)

# 모델 학습
ada_model.fit(x,y)


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

In [18]:
from sklearn.tree import DecisionTreeRegressor

# 결정트리 (max_depth3) 모델 구축 및 학습
tree_reg_model_1 = DecisionTreeRegressor(max_depth=3)
tree_reg_model_1.fit(x,y)

# 첫 번째 학습기에서 발생한 잔차를 목적함수로 모델 학습
residual_1 = y - tree_reg_model_1.predict(x)
tree_reg_model_2 = DecisionTreeRegressor(max_depth=3)
tree_reg_model_2.fit(x, residual_1)

# 두 번째 학습기에서 발생한 잔차를 목적함수로 모델 학습
residual_2 = y - tree_reg_model_2.predict(x)
tree_reg_model_3 = DecisionTreeRegressor(max_depth=3)
tree_reg_model_3.fit(x, residual_2)

# 새로운 데이터를 세 개의 트리를 폼한한 앙상블 모델로 예측
x_new = [[1.4,0.2]]
prediction = sum(tree.predict(x_new) for tree in [tree_reg_model_1, tree_reg_model_2, tree_reg_model_3])
prediction

array([-5.20417043e-18])

In [19]:
# 사이킷 런에서 제공하는 GBRT 앙상블을 다음과 같이 간단하게 훈련시킬 수 있다.

from sklearn.ensemble import GradientBoostingRegressor
# GBRT 모형 구축
# GBRT 앙상블 모형도 마찬가지로 n_estimator, max_deptj, min_samples_leaf 등을 통해 모델 규제가 가능하다
# 추가적으로 learning_rate가 각 트리의 기여 정도를 조절하다.
# learning_rate가 0.1보다 낮게 설정되면 훈련을 위한 트리가 더 많이 필요하지만 성능은 좋아진다.
# 이러한 방식을 축소(shrinkage)라고 부르는 규제 방법이다.
gbrt = GradientBoostingRegressor(max_depth = 3 ,
                                 n_estimators = 3,
                                 learning_rate = 1)
# GBRT 모형 학습
gbrt.fit(x, y)

GradientBoostingRegressor(learning_rate=1, n_estimators=3)